In [1]:
import pandas as pd
import numpy as np

In [2]:
val = pd.read_csv('../results/metrics/ihdp/tl_mlp/tl_mlp_val_metrics.csv')
test = pd.read_csv('../results/metrics/ihdp/tl_mlp/tl_mlp_test_metrics.csv')

In [9]:
val['mse'] = val[['mse_m0', 'mse_m1']].mean(axis=1)
test['mse'] = test[['mse_m0', 'mse_m1']].mean(axis=1)

In [10]:
val.head()

,iter_id,fold_id,param_id,mse_m0,mse_m1,ate,pehe,ate_hat,r2_score_m0,r2_score_m1,mse
0,1,1,1,0.401542,0.410144,0.395604,0.880498,3.502658,0.518008,-0.879962,0.405843
1,1,1,2,0.401542,0.361513,0.274447,0.792328,3.623815,0.518008,-0.657050,0.381527
2,1,1,3,0.401542,0.225341,0.230005,0.888396,3.668257,0.518008,-0.032887,0.313442
3,1,1,4,0.401542,0.322378,0.210146,0.913138,3.688116,0.518008,-0.477672,0.361960
4,1,1,5,0.401542,0.371681,0.241721,1.160247,3.656541,0.518008,-0.703660,0.386612


In [11]:
test.head()

,iter_id,param_id,mse_m0,mse_m1,ate,pehe,ate_hat,r2_score_m0,r2_score_m1,mse
0,1,1,0.38866,0.224202,0.416340,0.851057,3.614209,0.526041,-0.559335,0.306431
1,1,2,0.38866,0.234494,0.418472,0.909595,3.612077,0.526041,-0.630919,0.311577
2,1,3,0.38866,0.200377,0.207256,0.819748,3.823293,0.526041,-0.393633,0.294519
3,1,4,0.38866,0.206689,0.052043,0.889979,3.978506,0.526041,-0.437532,0.297675
4,1,5,0.38866,0.304307,0.316328,1.136573,3.714221,0.526041,-1.116471,0.346484


In [12]:
val_gr = val.groupby(['iter_id', 'param_id'], as_index=False).mean().drop(columns=['fold_id'])


In [13]:
merged = val_gr.merge(test, on=['iter_id', 'param_id'], suffixes=['_val', '_test'])

In [14]:
merged.head()

,iter_id,param_id,mse_m0_val,mse_m1_val,ate_val,pehe_val,ate_hat_val,r2_score_m0_val,r2_score_m1_val,mse_val,mse_m0_test,mse_m1_test,ate_test,pehe_test,ate_hat_test,r2_score_m0_test,r2_score_m1_test,mse_test
0,1,1,0.308468,0.332974,0.379937,0.885449,3.634514,0.366529,-0.346123,0.320721,0.38866,0.224202,0.416340,0.851057,3.614209,0.526041,-0.559335,0.306431
1,1,2,0.308468,0.297709,0.282704,0.857364,3.731748,0.366529,-0.185468,0.303088,0.38866,0.234494,0.418472,0.909595,3.612077,0.526041,-0.630919,0.311577
2,1,3,0.308468,0.313899,0.257049,0.901955,3.757403,0.366529,-0.223642,0.311183,0.38866,0.200377,0.207256,0.819748,3.823293,0.526041,-0.393633,0.294519
3,1,4,0.308468,0.344830,0.202597,0.943530,3.811855,0.366529,-0.342170,0.326649,0.38866,0.206689,0.052043,0.889979,3.978506,0.526041,-0.437532,0.297675
4,1,5,0.308468,0.353987,0.186844,0.991579,3.827608,0.366529,-0.376735,0.331227,0.38866,0.304307,0.316328,1.136573,3.714221,0.526041,-1.116471,0.346484


In [16]:
iter_gr = merged.groupby(['iter_id'], as_index=False)

In [30]:
cols = ['mse_val', 'ate_val', 'pehe_val', 'mse_test', 'ate_test', 'pehe_test']

best_mse = iter_gr.apply(lambda x: x.loc[x['mse_val'].idxmin(), cols])
test_mse = iter_gr.apply(lambda x: x.loc[x['mse_test'].idxmin(), cols])
best_ate = iter_gr.apply(lambda x: x.loc[x['ate_test'].idxmin(), cols])
best_pehe = iter_gr.apply(lambda x: x.loc[x['pehe_test'].idxmin(), cols])

In [27]:
best_mse.head(10)

,iter_id,mse_val,ate_val,pehe_val,mse_test,ate_test,pehe_test
0,1,0.255598,0.166683,0.523266,0.251229,0.259597,0.550521
1,2,0.275102,0.122573,0.597211,0.213787,0.046736,0.704321
2,3,0.226245,0.082278,0.660801,0.214572,0.115020,0.715075
3,4,0.149174,0.094865,0.841978,0.188866,0.006365,0.694214
4,5,0.108651,0.184372,0.664468,0.128001,0.071471,0.664003
5,6,0.213941,0.127536,0.643121,0.345081,0.162974,0.664032
6,7,0.326584,0.371576,0.581334,0.371363,0.369198,0.548655
7,8,0.199583,0.246551,0.780199,0.200302,0.218490,0.836494
8,9,0.024587,0.706044,4.679700,0.007030,0.835723,2.483899
9,10,0.022226,0.308851,1.376793,0.024134,0.120544,1.075899


In [28]:
best_pehe.head(10)

,iter_id,mse_val,ate_val,pehe_val,mse_test,ate_test,pehe_test
0,1,0.309212,0.315807,0.654496,0.270331,0.087123,0.406877
1,2,0.311272,0.134204,0.810567,0.199337,0.059812,0.267377
2,3,0.269961,0.425974,0.741955,0.179692,0.212911,0.412818
3,4,0.166545,0.090261,0.840886,0.174780,0.010776,0.451119
4,5,0.116391,0.193064,0.687381,0.125031,0.014932,0.648115
5,6,0.249110,0.103773,0.717481,0.349214,0.077135,0.265793
6,7,0.339693,0.361499,0.540772,0.363224,0.239621,0.430966
7,8,0.246896,0.322438,0.890087,0.206068,0.014620,0.498904
8,9,0.024822,0.611954,4.626936,0.006886,0.507029,2.300411
9,10,0.024565,0.255408,1.263560,0.023970,0.089433,0.728904


In [31]:
test_mse.head(10)

,iter_id,mse_val,ate_val,pehe_val,mse_test,ate_test,pehe_test
0,1,0.288961,0.155441,0.737541,0.195786,0.232579,0.971439
1,2,0.314756,0.120113,0.821286,0.148317,0.022253,0.673070
2,3,0.264275,0.451865,0.724332,0.148280,0.280682,0.497836
3,4,0.182850,0.403414,0.842859,0.132097,0.615556,0.831246
4,5,0.141914,0.231234,1.123406,0.113502,0.274882,0.907996
5,6,0.245940,0.125911,0.869474,0.274077,0.193532,1.009173
6,7,0.406369,0.207575,0.907729,0.263627,0.316927,0.930756
7,8,0.222924,0.224473,0.796197,0.153833,0.326456,0.724743
8,9,0.024996,0.718082,4.699039,0.006542,0.674503,2.486808
9,10,0.025689,0.231661,1.398473,0.020467,0.009671,0.878144


In [29]:
best_ate.head(10)

,iter_id,mse_val,ate_val,pehe_val,mse_test,ate_test,pehe_test
0,1,0.332496,0.108428,0.948651,0.391466,0.000011,0.940277
1,2,0.303029,0.164079,0.721408,0.175967,0.000024,0.703449
2,3,0.255226,0.130811,0.850232,0.186018,0.000064,0.604307
3,4,0.184797,0.123516,0.962522,0.165966,0.000023,0.789473
4,5,0.145310,0.117907,1.081868,0.177542,0.000017,1.250277
5,6,0.255069,0.194790,0.870089,0.385849,0.000029,0.943013
6,7,0.394798,0.194120,0.795227,0.432596,0.000062,0.762857
7,8,0.270443,0.186112,1.244313,0.312576,0.000101,1.330902
8,9,0.078213,0.762014,9.297575,0.045280,0.000046,7.058912
9,10,0.039118,0.338771,1.811503,0.031852,0.000014,1.409989


In [52]:
merged[merged['iter_id'] == 1][cols].corr()

,mse_val,ate_val,pehe_val,mse_test,ate_test,pehe_test
mse_val,1.000000,-0.098748,0.795228,0.392761,-0.251548,0.485091
ate_val,-0.098748,1.000000,-0.215477,-0.327883,0.511143,-0.316249
pehe_val,0.795228,-0.215477,1.000000,0.470914,-0.263375,0.673839
mse_test,0.392761,-0.327883,0.470914,1.000000,-0.314174,0.624225
ate_test,-0.251548,0.511143,-0.263375,-0.314174,1.000000,-0.107524
pehe_test,0.485091,-0.316249,0.673839,0.624225,-0.107524,1.000000


In [53]:
corrs = np.zeros((10, 6, 6))
for i in range(10):
    corrs[i] = merged[merged['iter_id'] == i+1][cols].corr()

In [57]:
pd.DataFrame(corrs.mean(axis=0), columns=cols, index=cols)

,mse_val,ate_val,pehe_val,mse_test,ate_test,pehe_test
mse_val,1.000000,0.147620,0.846393,0.492142,0.038606,0.654970
ate_val,0.147620,1.000000,0.106209,0.073690,0.400091,0.034438
pehe_val,0.846393,0.106209,1.000000,0.502135,0.033672,0.745821
mse_test,0.492142,0.073690,0.502135,1.000000,0.107460,0.679192
ate_test,0.038606,0.400091,0.033672,0.107460,1.000000,0.237656
pehe_test,0.654970,0.034438,0.745821,0.679192,0.237656,1.000000
